### (0) IMPORT

In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

#BORROWING KAISHENGS CLIENT SECRET AND ID THANKS KS BB PLS DONT DISABLE YA
auth_manager = SpotifyClientCredentials(client_id="21ff73a9b5a94ea8b3a969b906baead1", client_secret="3761e7947ef542149467196a07cf2563")
sp = spotipy.Spotify(auth_manager=auth_manager)

### (1) Playlist extractor
Make sure you do not already have a file called "User_Playlist.csv". if you do, please delete it if not this code will encounter an error

In [18]:
import time

# def extract_playlistID(playlists,find_playlist):
#     while True:
#         for playlist in playlists['items']:
#         ##Loop and return playlistID if found
#             if playlist["name"] == find_playlist:
#                 return playlist['id']
#         ##Show next 50 songs
#         if playlists['next']:
#             playlists = sp.next(playlists)
#         else:
#             return None
        
def extract_trackID(playlist_tracks):
    trackID_list = []
    for tracks in playlist_tracks["items"]:
        track_ID = tracks["track"]["id"]
        trackID_list.append(track_ID)
    return trackID_list

def extract_nexttrackID(playlist_tracks):
    trackID_list = []
    for tracks in playlist_tracks["items"]:
        track_ID = tracks["track"]["id"]
        trackID_list.append(track_ID)
    return trackID_list

def extract_songInfo(song):
    info = sp.track(song)
    ##Define features u wanna extract here
    id = song
    songName = info["name"]
    albumName = info["album"]["name"]
    albumID = info["album"]["id"]
    artists = info["artists"][0]["name"]
    for artist in info["artists"]:
        artist_genre = sp.artist(artist["id"])
        genre = artist_genre["genres"]
    explicit = info["explicit"]
    releaseDate = info["album"]["release_date"]
    return [id,songName,albumName,albumID,artists,genre,explicit,releaseDate]

def extract_features(song):
    info = sp.audio_features(song)[0]
    ##Define features u wanna extract here
    danceability = info["danceability"]
    energy = info["energy"]
    key = info["key"]
    loudness = info["loudness"]
    mode = info["mode"]
    speechiness = info["speechiness"]
    acousticness = info["acousticness"]
    instrumental = info["instrumentalness"]
    liveness = info["liveness"]
    valence = info["valence"]
    tempo = info["tempo"]
    duration_ms = info["duration_ms"]
    time_signature = info["time_signature"]
    return [danceability,energy,key,loudness,mode,speechiness,acousticness,instrumental,liveness,
            valence,tempo,duration_ms,time_signature]

#Auth Stuff
auth_manager = SpotifyClientCredentials(client_id="21ff73a9b5a94ea8b3a969b906baead1", client_secret="3761e7947ef542149467196a07cf2563")
sp = spotipy.Spotify(auth_manager=auth_manager)
    
playlists_name = []
playlists_ID = []

## Ask to insert playlist ID
while True:
    url = input("Insert Spotify Playlist ID (Else type 'done'): ")
    if url == "done":
        break
    else:
        playlists_ID.append(url)
    
# Get Playlist Name
print("Obtaining the Following Playlists: ")
for i in playlists_ID:
    playlist_info = sp.playlist(i)
    name = playlist_info["name"]
    print(name)
    playlists_name.append(name)

##Get Info in Playlist with PlaylistID 
tracksID = {}
for i in range(len(playlists_ID)):
    track_list = []
    playlist_tracks = sp.playlist(playlists_ID[i])
    playlist_tracks = playlist_tracks["tracks"]
    track_list = extract_trackID(playlist_tracks)
    for page in range(0,2):
        if playlist_tracks['next']:
            print("More than 100 Songs Detected. WAITT")
            time.sleep(30)
            print("Looking at next 100 songs")
            playlist_tracks = sp.next(playlist_tracks)
            track_list += extract_nexttrackID(playlist_tracks)
    tracksID[playlists_name[i]] = track_list


##Get Other Track Details
results_tocsv = []
results_tocsv.append(["id","songName","albumName","albumID","artist","genre","explicit","releaseDate",
                     "danceability","energy","key","loudness","mode","speechiness","acousticness","instrumental","liveness",
                    "valence","tempo","duration_ms","time_signature"])
for i in playlists_name:  
    count = 0
    for song in tracksID[i]:
        time.sleep(1)
        song_info = extract_songInfo(song)
        ##Get Audio Features
        song_features = extract_features(song)
        song_info.extend(song_features)
        results_tocsv.append(song_info)
        count += 1
        
        if count == 100:
            print("Trying not to let spotify kill me. 30s wait :D")
            time.sleep(30)
            count = 0

##Use pandas to convert to dataframe
df = pd.DataFrame(results_tocsv[1:], columns = results_tocsv[0])

##Export to CSV
file_name = "lany.csv"
df.to_csv(file_name, encoding='utf_8_sig')
print("20s Cooldown...")
time.sleep(20)

Obtaining the Following Playlists: 
This Is LANY
20s Cooldown...


### (2) FUNCTIONSSS

In [2]:
def ohe_prep(df, column, new_name): 
    ''' 
    Create One Hot Encoded features of a specific column
    ---
    Input: 
    df (pandas dataframe): Spotify Dataframe
    column (str): Column to be processed
    new_name (str): new column name to be used
        
    Output: 
    tf_df: One-hot encoded features 
    '''
    
    tf_df = pd.get_dummies(df[column])
    
    feature_names = tf_df.columns
    tf_df.columns = [new_name + "|" + str(i) for i in feature_names]
    tf_df.reset_index(drop = True, inplace = True)  
    return tf_df

def get_features_database(databaseDF):
    #Select Features
    databaseDF = databaseDF[["id","songName",
                "danceability","energy","key","loudness","mode","speechiness","acousticness","instrumental","liveness",
                "valence","tempo","type"]]

    #OHE Features
    key_ohe = ohe_prep(databaseDF, 'key','key') * 0.5
    mode_ohe = ohe_prep(databaseDF, 'mode','mode') * 0.5

    ##Normalise/Scale Audio Columns
    float_cols = databaseDF.dtypes[databaseDF.dtypes == 'float64'].index.values
    floats = databaseDF[float_cols].reset_index(drop = True)
    scaler = MinMaxScaler()
    floats_scaled = pd.DataFrame(scaler.fit_transform(floats), columns = floats.columns) * 0.2

    ##Combine all Features
    final = pd.concat([floats_scaled, key_ohe, mode_ohe, databaseDF["type"]], axis = 1)
    return final

### (3) GENERALISING/SIMPLIFYING Genre Name of Input Songs to Match Kaggle DS for recommendation
Reduces the highly specific genres of Input Songs retrieved spotipy to increase match with kaggle ds with generalised genres (pre cleaned, retrieved from web)
If METHOD1 does not work for particular song, do METHOD2 so on and so forth <br><br>
METHOD1 - Find User Songs in Dataset and Copy Over <br>
METHOD2 - Webscrap from get_genre.com which genre name is generic and similar to dataset<br>


### (3.1) Importing Web driver and BeautifulSoup

METHOD2 - Webscraping Function: REQUIRES SELENIUM AND BS4 LIBRARY. FOR SELENIUM NEEDS TO INSTALL CHROMEDRIVERS TOO from below: <br>


[CHROMEDRIVERS](https://googlechromelabs.github.io/chrome-for-testing) 

In [5]:
from bs4 import BeautifulSoup
from selenium import webdriver
import time

genre_dict = {
    "pop": ["pop punk", "pop rap", "pop rock", "synthpop","dance-pop", "singer-songwriter"],
    "rock": ["psych rock","power pop","alt rock", "hard rock","emo","blues", "folk", "garage", "pop rock", "acoustic", "rock and roll", "singer-songwriter", ""],
    "metal": ["metalcore","black metal", "death metal", "heavy metal", "goth", "gothic metal", "groove", "punk", "grindcore", "industrial", "alternative metal"],
    "house": ["chicago house", "progressive house", "deep house", "edm", "electro house", "future house", "tech house"],
    "country": ["country pop", "country blues", "country rap", "country rock"],
    "r&b": ["soul","dance","contemporary r&b", "alternative r&b", "soul jazz", "r&b and soul","rhythm and blues"],
    "techno": ["detroit techno","minimal techno", "hardcore"],
    "electro": ["drum and bass","dubstep","electronic", "club", "electro house", "electronic dance music", "electro swing", "electropop"],
    "hip hop": ["rap", "funk", "alternative hip hop", "rap rock", "rap metal", "jazz rap"],
    "k-pop": [],
    "indie": ["indie rock", "indie pop"]
}


def generaliseGenre(genre):
    for key,value in genre_dict.items():
        ##if genre scrapped is same as generic genre(key), return genre
        if key == genre:
            return key
        else:
            ##iterate through all the genres (value) in the dict to find suitable generic genre
            for alt_genre in value:
                if alt_genre == genre:
                    return key
    ##return none if no genre found
    return None

def scrap_genre(artistName):

    ##URL of website to scrap
    url = f"https://www.getgenre.com/artist/{artistName}"

    ##CHANGE PATH TO PATH OF DOWNLOADED CHROMEDRIVER
    driver = webdriver.Chrome(executable_path="/Users/kaisheng/Documents/chromedriver")

    # WAIT FOR BUTTON TAG TO LOAD BEFORE CONTINUING
    driver.implicitly_wait(30)
    driver.get(url)

    classtext = "MuiButtonBase-root MuiButton-root MuiButton-outlined MuiButton-outlinedPrimary MuiButton-sizeMedium MuiButton-outlinedSizeMedium MuiButton-root MuiButton-outlined MuiButton-outlinedPrimary MuiButton-sizeMedium MuiButton-outlinedSizeMedium css-x3ahaf"
    button = driver.find_element_by_xpath(f"//button[contains(@class, '{classtext}')]")
    time.sleep(2)
    # Parse the HTML content of the button with BS4
    html = button.get_attribute("outerHTML")
    driver.close()
    driver.quit()

    #Use BS4 to read HTML segement and extract Genre Text
    soup = BeautifulSoup(html, "html.parser")
    try:
        # Find html button tag with class to obtained genre text
        genre = soup.find('button', class_=classtext).get_text().lower()
        result = generaliseGenre(genre)
        return result
    except:
        pass


In [6]:
##Import User Playlist
userDF = pd.read_csv("User_Playlist.csv")
userDF = userDF.drop(columns=["Unnamed: 0"])
##Import Dataset
df = pd.read_csv("spotify_data_cleaned.csv")
df = df.drop(columns=["Unnamed: 0"])
df = df.dropna()

#Extract all genres from dataset
datasetGenres_list = []
for i in df["genre"]:
    if i not in datasetGenres_list:
        datasetGenres_list.append(i)

song_genres_list = []

## Dict for scrapped genres to avoid scraping same artist multiple times
scrap_dict = {}

##Find Playlist Songs in Database and replace genre
for index, row in userDF.iterrows():
    id = row["id"]
    artistName = row["artist"]

    ##List to include all genres from one song
    one_song_genre = []

    ##Check if artistName is already scrapped or tracked
    if artistName in scrap_dict.keys():
        genre = scrap_dict[artistName]
        one_song_genre.append(genre)
    else:
        ##variable to check if song in database
        dataset_genre = df[df["id"] == id]

        if not dataset_genre.empty:
            ##METHOD1 - Searches for kaggle ds for song inputted to retrieve genre (matching kaggle ds). 
        ##If found, genre from kaggle ds replaces highly specific genre of song retrieved from spotify
            one_song_genre = dataset_genre["genre"].to_list()
            for i in one_song_genre:
                scrap_dict[artistName] = i
        else:
            ##METHOD2 - Web Scraping to retrieve simplified genre from getgenre.com to replace highly specific genre retrieved from spotify
            genre = scrap_genre(artistName)
            scrap_dict[artistName] = genre
            if genre != "":
                one_song_genre.append(genre)

    ##CHECKS IF METHOD 1 or 2 managed to obtain generalised genre data
    song_genres_list.append(one_song_genre)

userDF["genre"] = song_genres_list
userDF

,id,songName,albumName,albumID,artist,genre,explicit,releaseDate,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumental,liveness,valence,tempo,duration_ms,time_signature
0,24yrJJkmVUMkJHvCaR85BN,Snooze,Snooze,0L632gxhMbscW5mY9ewbOq,SZA,[r&b],True,2023-08-25,0.571,0.5520,...,-7.231,1,0.1300,0.1400,0.000000,0.1090,0.3930,143.024,201800,4
1,0otRX6Z89qKkHkQ9OqJpKt,Car's Outside,YOU,0cm9LOQUBK3JUgyoRj238z,James Arthur,[pop],False,2019-10-18,0.338,0.5620,...,-7.181,1,0.0290,0.0390,0.000000,0.1070,0.2590,149.907,248373,4
2,6ErCfbbI9BG4bK2djjKUvP,i wish you cheated,i wish you cheated,1tf89mHJkN2HVQ4xmdUUqy,Alexander Stewart,[pop],False,2023-08-09,0.348,0.3810,...,-7.688,1,0.0552,0.2040,0.000000,0.1330,0.4940,84.896,190352,3
3,3vZk7OAUjMtVDNC852aNqi,Oceans & Engines,Oceans & Engines,2va673nk2JXgCxJeTiZdM9,NIKI,[r&b],False,2022-07-08,0.395,0.3090,...,-8.919,1,0.0330,0.6370,0.000022,0.0931,0.0673,142.012,336001,3
4,1kuGVB7EU95pJObxwvfwKS,vampire,GUTS,1xJHno7SmdVtZAtXbdbDZp,Olivia Rodrigo,[pop],True,2023-09-08,0.511,0.5320,...,-5.745,1,0.0578,0.1770,0.000000,0.2910,0.3500,138.005,219724,4
5,6wf7Yu7cxBSPrRlWeSeK0Q,What Was I Made For? [From The Motion Picture ...,What Was I Made For? [From The Motion Picture ...,3AafSrFIbJPH6BJHiJm1Cd,Billie Eilish,[pop],False,2023-07-13,0.444,0.0911,...,-17.665,1,0.0307,0.9590,0.000001,0.0980,0.1420,78.403,222370,4
6,3GYlZ7tbxLOxe6ewMNVTkw,TV,Guitar Songs,1YPWxMpQEC8kcOuefgXbhj,Billie Eilish,[pop],False,2022-07-21,0.412,0.2360,...,-15.159,0,0.0379,0.8300,0.006530,0.1470,0.1190,140.214,281381,4
7,7hDVYcQq6MxkdJGweuCtl9,ocean eyes,dont smile at me,7fRrTyKvE4Skh93v97gtcU,Billie Eilish,[pop],False,2017-12-22,0.511,0.3630,...,-7.650,0,0.0410,0.8160,0.031700,0.0840,0.1690,144.892,200379,4
8,4VuS959DSpr82t3qBqCrWG,Supermarket Flowers,÷ (Deluxe),3T4tUhGYeRNVUGevb0wThu,Ed Sheeran,[pop],False,2017-03-03,0.589,0.2420,...,-10.517,1,0.0442,0.9140,0.000000,0.0887,0.2570,89.749,221107,4
9,1ZMiCix7XSAbfAJlEZWMCp,Falling,Fine Line,7xV2TzoaVc0ycW7fwBwAml,Harry Styles,[pop],False,2019-12-13,0.567,0.2670,...,-6.502,1,0.0299,0.8390,0.000001,0.0890,0.0592,110.011,240133,4


### (4) Find Top 3 Genres in User Playlist By Counting Frequency of Genres

In [7]:
# Find top 3 genres in playlist
exploded_df = userDF.explode('genre')

freq = exploded_df["genre"].value_counts(sort=True)

genres_count = dict(freq.head(3))
genre_top3 = list(genres_count.keys())
        
print("Top 3 Genre's of User Playlist:" + str(genre_top3))
exploded_df

Top 3 Genre's of User Playlist:['pop', 'r&b', 'rock']


,id,songName,albumName,albumID,artist,genre,explicit,releaseDate,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumental,liveness,valence,tempo,duration_ms,time_signature
0,24yrJJkmVUMkJHvCaR85BN,Snooze,Snooze,0L632gxhMbscW5mY9ewbOq,SZA,r&b,True,2023-08-25,0.571,0.5520,...,-7.231,1,0.1300,0.1400,0.000000,0.1090,0.3930,143.024,201800,4
1,0otRX6Z89qKkHkQ9OqJpKt,Car's Outside,YOU,0cm9LOQUBK3JUgyoRj238z,James Arthur,pop,False,2019-10-18,0.338,0.5620,...,-7.181,1,0.0290,0.0390,0.000000,0.1070,0.2590,149.907,248373,4
2,6ErCfbbI9BG4bK2djjKUvP,i wish you cheated,i wish you cheated,1tf89mHJkN2HVQ4xmdUUqy,Alexander Stewart,pop,False,2023-08-09,0.348,0.3810,...,-7.688,1,0.0552,0.2040,0.000000,0.1330,0.4940,84.896,190352,3
3,3vZk7OAUjMtVDNC852aNqi,Oceans & Engines,Oceans & Engines,2va673nk2JXgCxJeTiZdM9,NIKI,r&b,False,2022-07-08,0.395,0.3090,...,-8.919,1,0.0330,0.6370,0.000022,0.0931,0.0673,142.012,336001,3
4,1kuGVB7EU95pJObxwvfwKS,vampire,GUTS,1xJHno7SmdVtZAtXbdbDZp,Olivia Rodrigo,pop,True,2023-09-08,0.511,0.5320,...,-5.745,1,0.0578,0.1770,0.000000,0.2910,0.3500,138.005,219724,4
5,6wf7Yu7cxBSPrRlWeSeK0Q,What Was I Made For? [From The Motion Picture ...,What Was I Made For? [From The Motion Picture ...,3AafSrFIbJPH6BJHiJm1Cd,Billie Eilish,pop,False,2023-07-13,0.444,0.0911,...,-17.665,1,0.0307,0.9590,0.000001,0.0980,0.1420,78.403,222370,4
6,3GYlZ7tbxLOxe6ewMNVTkw,TV,Guitar Songs,1YPWxMpQEC8kcOuefgXbhj,Billie Eilish,pop,False,2022-07-21,0.412,0.2360,...,-15.159,0,0.0379,0.8300,0.006530,0.1470,0.1190,140.214,281381,4
7,7hDVYcQq6MxkdJGweuCtl9,ocean eyes,dont smile at me,7fRrTyKvE4Skh93v97gtcU,Billie Eilish,pop,False,2017-12-22,0.511,0.3630,...,-7.650,0,0.0410,0.8160,0.031700,0.0840,0.1690,144.892,200379,4
8,4VuS959DSpr82t3qBqCrWG,Supermarket Flowers,÷ (Deluxe),3T4tUhGYeRNVUGevb0wThu,Ed Sheeran,pop,False,2017-03-03,0.589,0.2420,...,-10.517,1,0.0442,0.9140,0.000000,0.0887,0.2570,89.749,221107,4
9,1ZMiCix7XSAbfAJlEZWMCp,Falling,Fine Line,7xV2TzoaVc0ycW7fwBwAml,Harry Styles,pop,False,2019-12-13,0.567,0.2670,...,-6.502,1,0.0299,0.8390,0.000001,0.0890,0.0592,110.011,240133,4


### (5) Extract Both User and Dataset Csv

In [19]:
#Extract Database CSV
databaseDF = pd.read_csv("spotify_dataset_eda.csv", encoding="utf_8_sig")
databaseDF = databaseDF.drop(columns=['Unnamed: 0'])
#Selects columns that we want
databaseDF = databaseDF[["id","artist","songName", "genre","danceability","energy","key","loudness","mode","speechiness","acousticness","instrumental","liveness",
        "valence","tempo",]]

#Create new feature/column of artist_songName
databaseDF["artist_songName"] = databaseDF["artist"] + "_" + databaseDF["songName"]

##Change all values to lowercase
databaseDF["artist_songName"] = databaseDF["artist_songName"].str.lower()

#Check for duplicates in database
databaseDF = databaseDF.drop_duplicates(subset=["artist_songName"],ignore_index= True)

In [21]:
#Extract User Playlist CSV
playlistDF = pd.read_csv("User_Playlist.csv", encoding="utf_8_sig")

playlistDF = playlistDF.drop(columns=['Unnamed: 0'])
#Reset Index
playlistDF = playlistDF.reset_index(drop = True)  
#Selects columns that we want
playlistDF = playlistDF[["id","artist","songName","danceability","energy","key","loudness","mode","speechiness","acousticness","instrumental","liveness",
        "valence","tempo",]] 

#Create new feature/column of artist_songName to remove duplicate later on
playlistDF["artist_songName"] = playlistDF["artist"] + "_" + playlistDF["songName"]
##Change all values to lowercase
playlistDF["artist_songName"] = playlistDF["artist_songName"].str.lower()

#Check for duplicates in user Playlist
playlistDF = playlistDF.drop_duplicates(subset=["artist_songName"],ignore_index= True)

### (6) Normalise Features

In [22]:
#Merge user + dataset dataframe to normalise 
#Normalise takes min and max in dataframe as reference and change it to 0 and 1 respectively

#Group the dataframe as we gonna split it again alter
databaseDF["type"] = "Dataset"
playlistDF["type"] = "User"


#Merge the 2 datasets together
combinedDF = pd.concat([databaseDF,playlistDF], ignore_index=True)

#Check for duplicates between user and Database
combinedDF = combinedDF.drop_duplicates(subset=["artist_songName"], keep="last",ignore_index= True)

##Update databaseDF with removed songs from user Playlist
databaseDF = combinedDF[(combinedDF["type"]== "Dataset")]

##Normalise and get Vectors for Dataset + User
normalised_vector = get_features_database(combinedDF)

##Seperate User from databaseDF 
database_vector = normalised_vector[normalised_vector["type"] == "Dataset"]
user_vector = normalised_vector[normalised_vector["type"] == "User"]

#Drop "type" column
database_vector = database_vector.drop(columns="type")
user_vector = user_vector.drop(columns="type")
databaseDF = databaseDF.drop(columns=["type"])


### (7) Merge all User Vector to form single vector (this is where we gotta do sth w clustering model to not squeeze every song vector into one ball of mess bc the recos are so shit)

In [23]:
##Single Vector Creation
final_user_vector_list = []
for i in user_vector.columns:
    final_user_vector_list.append(user_vector[i].sum()/len(user_vector[i]))

#Putting into a vector dataframe
final_user_vector = pd.DataFrame(columns=user_vector.columns,)
final_user_vector.loc[0] = final_user_vector_list

final_user_vector

,danceability,energy,loudness,speechiness,acousticness,instrumental,liveness,valence,tempo,key|0,...,key|4,key|5,key|6,key|7,key|8,key|9,key|10,key|11,mode|0,mode|1
0,0.10342,0.084711,0.146276,0.011244,0.10182,0.00039,0.024032,0.058496,0.094853,0.116071,...,0.053571,0.017857,0.035714,0.026786,0.017857,0.044643,0.035714,0.071429,0.0625,0.4375


### (LAST) RECOMMEND TIME!
if the database songs that have high cosine similarity as input song DO NOT SHARE SAME GENRE (ALL FROM SPOTIFY BTW), THEN WE WILL GET NO RECOS REGARDLESS

SO if the results is empty, it simply means the kaggle ds does not contain songs of that genre

In [32]:
def generate_rec(databaseDF, database_vector, user_vector, genre_top3):
    #Cosine Similarity
    databaseDF["sim"] = cosine_similarity(database_vector,user_vector)
    # Drop rows with different genre from top 3 genres
    if len(genre_top3) == 1:
        datasetChunk = databaseDF[(databaseDF["genre"] == genre_top3[0])]
    elif len(genre_top3) == 2:
        datasetChunk = databaseDF[(databaseDF["genre"] == genre_top3[0]) | (databaseDF["genre"] == genre_top3[1])]
    else:
        datasetChunk = databaseDF[(databaseDF["genre"] == genre_top3[0]) | (databaseDF["genre"] == genre_top3[1]) | (databaseDF["genre"] == genre_top3[2])]
    
    ##Check if user genre is in database
    if datasetChunk.empty:
        return databaseDF.sort_values('sim',ascending = False).head()
    
    rec_sorted = datasetChunk.sort_values('sim',ascending = False)


    # ##Web Scrap Dataset
    # rec_top5 = pd.DataFrame(columns=rec_sorted.columns)
    # for index, row in rec_sorted.iterrows():    
    #     artist = row["artist"]
    #     genre = scrap_genre(artist)
    #     rec_top5 = rec_top5.dropna(axis=1, how='all')
    #     if genre != "" and genre in genre_top3: 
    #         rec_top5 = pd.concat([rec_top5, pd.DataFrame([row])], ignore_index=True)
    #     if len(rec_top5) == 5:
    #         return rec_top5
        
            
    #Sort and recommend top 5 with same genres
    rec_top5 = rec_sorted.head(10)
    return rec_top5

In [33]:
##Generate Recc Songs
result = generate_rec(databaseDF,database_vector,final_user_vector, genre_top3)
result

,id,artist,songName,genre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumental,liveness,valence,tempo,artist_songName,sim
204305,4x4cclsdcnzWphsjJzD4Bs,Tierra Whack,Silly Sam,r&b,0.765,0.488,0,-4.412,1,0.0780,0.710,0.000000,0.1580,0.502,179.874,tierra whack_silly sam,0.854393
230906,4fj7e6Tc48Bmomwyz2Z2XC,Manuel Turizo,Te Quemaste,pop,0.729,0.773,0,-2.486,1,0.1310,0.611,0.001060,0.2870,0.718,176.053,manuel turizo_te quemaste,0.853823
427348,1e0xat8klVTJg7OnaufrNr,The Moldy Peaches,County Fair,rock,0.728,0.632,0,-3.129,1,0.0658,0.712,0.000000,0.0551,0.442,125.458,the moldy peaches_county fair,0.852978
451816,2wsXXPcycZRzfTGKF4HS4T,22-20s,I'm Your Witchdoctor,rock,0.546,0.762,0,-3.242,1,0.0402,0.738,0.000694,0.2230,0.721,177.559,22-20s_i'm your witchdoctor,0.852522
30481,0M4OYuJUxu4fnJ2cRrAkEk,Human Petting Zoo,Anxiety Song,rock,0.713,0.526,0,-3.345,1,0.0532,0.940,0.000000,0.1260,0.660,128.350,human petting zoo_anxiety song,0.852394
33729,2DpQtdYv2OJ0AiPWAKIl4t,The Sonics,Don't You Just Know It,rock,0.673,0.793,0,-6.172,1,0.1210,0.795,0.000068,0.2400,0.531,125.271,the sonics_don't you just know it,0.852234
299556,4GnyY54Ja4zx2GNDUCNZqB,JJ Heller,The Bare Necessities,rock,0.597,0.668,0,-3.478,1,0.2080,0.679,0.000000,0.2300,0.765,201.769,jj heller_the bare necessities,0.852124
249708,3b5xqoWMD6W0oW814guCqs,State Champs,Crying out Loud,rock,0.644,0.522,0,-5.015,1,0.0332,0.738,0.000000,0.1020,0.594,147.092,state champs_crying out loud,0.851873
187027,1SqtvHdK7YCSPFJLZ0a3ZD,Vance Joy,Bonnie & Clyde,rock,0.715,0.611,0,-5.448,1,0.0367,0.893,0.011300,0.0968,0.412,117.870,vance joy_bonnie & clyde,0.851849
72194,0f3SEk5tBgiIE6B9nxiVcN,This Wild Life,Looking Back,rock,0.583,0.580,0,-3.974,1,0.0328,0.685,0.000000,0.1820,0.410,144.947,this wild life_looking back,0.851828
